In [25]:
from sklearn.datasets import make_classification
import requests
import asyncio
import aiohttp
import numpy as np
from time import time

SERVER = "http://localhost:8000"

Запустим последовательно обучение

In [ ]:
X, y = make_classification(
    n_samples=1500000,   
    n_features=50,       
    n_informative=40,
    n_redundant=10,
    random_state=42
)
X = X.tolist()
y = y.tolist()
start_ = time()
response_1 = requests.post(f"{SERVER}/fit", json={"name": "model_rand_seq_1", "X": X, "y": y, "model_type": "randf"})
print("randomforesct ",time()-start_)
start = time()
params = {
    "solver": "saga",
    "penalty": "l1",
    "max_iter": 10_000,
    "C": 0.1,
    "warm_start": True,
    "n_jobs": -1
}
response_2 = requests.post(f"{SERVER}/fit", json={"name": "model_log_seq_1", "X": X, "y": y, "model_type": "logreg", "params":params})
print ("logistic_regression", time()-start)
print("all finished", time()-start_)
print(response_1.status_code, response_1.text)
print(response_2.status_code, response_2.text)

randomforesct  85.15872097015381
logistic_regression 60.16963005065918
all finished 145.32943105697632
200 {"status":"started"}
200 {"status":"started"}


Теперь запустим асинхронное обучение, видим что время выполнения уменьшилось

In [ ]:
start_ = time()
async with aiohttp.ClientSession() as s:
        response_1, response_2 = await asyncio.gather(
            s.post(f"{SERVER}/fit", json={"name": "model_rand_1", "X": X, "y": y, "model_type": "randf"}),
            s.post(f"{SERVER}/fit", json={"name": "model_log_1", "X": X, "y": y, "model_type": "logreg",
                                                      "params": params})
        )
        print(await response_1.json())
        print(await response_2.json())
print(time()-start_)

{'status': 'started'}
{'status': 'started'}
9.488520860671997


Теперь предскажем данные на обученнных моделях

In [71]:
response_1 = requests.post(f"{SERVER}/load", json={"name": "model_rand_1"})
response_2 = requests.post(f"{SERVER}/load", json={"name": "model_log_1"})
print(response_1.status_code, response_1.text)
print(response_2.status_code, response_2.text)

200 {"status":"loaded"}
200 {"status":"loaded"}


In [41]:
X, y = make_classification(
    n_samples=150000,   
    n_features=50,       
    n_informative=40,
    n_redundant=10
)
X = X.tolist()
y = y.tolist()


In [42]:
async with aiohttp.ClientSession() as s:
    response_1, response_2 = await asyncio.gather(
        s.post(f"{SERVER}/predict", json={"name": "model_rand_1", "X": X, "y": y, "model_type": "randf"}),
        s.post(f"{SERVER}/predict", json={"name": "model_log_1", "X": X, "y": y, "model_type": "logreg", "params": params})
    )

    print(await response_1.json())
    print(await response_2.json())


{'predictions': [0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [49]:
X = np.random.rand(10000, 5).tolist()
y = [0 if sum(row) < 2 else 1 for row in X]

Краевой случай когда имя модели уже использовлось ранее

In [94]:
response = requests.post(f"{SERVER}/fit", json={"name": "model_log_regression_1", "X": X, "y": y, "model_type": "logreg"})
print(response.text)


{"status":"started"}


In [74]:
response = requests.post(f"{SERVER}/fit", json={"name": "model_log_regression_1", "X": X, "y": y, "model_type": "logreg"})
print(response.text)

{"detail":"Model already exists."}


Проверка корректности работы функции remove

In [ ]:
response = requests.post(f"{SERVER}/fit", json={"name": "model_linear_regression_3", "X": X, "y": y, "model_type": "lr"})
print(response.text)

{"status":"started"}


In [76]:
response = requests.post(f"{SERVER}/remove", json={"name": "model_linear_regression_3"})
print(response.text)

{"status":"deleted"}


Проверка корректности работы функции load(загрузка не сущетсвующей модели)

In [77]:
response = requests.post(f"{SERVER}/load", json={"name": "model_linear_regression_3"})
print(response.text)

{"detail":"Model model_linear_regression_3 not found."}


In [79]:
X, y = make_classification(
    n_samples=1500,   
    n_features=50,       
    n_informative=40,
    n_redundant=10
)
X = X.tolist()
y = y.tolist()

In [80]:
response = requests.post(f"{SERVER}/fit", json={"name": "model_rand_2", "X": X, "y": y, "model_type": "randf"})
print(response.text, response.status_code)

{"status":"started"} 200


Проверка корректности работы predict(использование модели, которая не была загружена на inference)

In [82]:
response = requests.post(f"{SERVER}/predict", json={"name": "model_rand_2", "X": X, "y": y, "model_type": "randf"})
print(response.text, response.status_code)

{"detail":"Model not loaded."} 400


In [83]:
response = requests.post(f"{SERVER}/load", json={"name": "model_rand_2"})
print(response.text, response.status_code)

{"status":"loaded"} 200


In [86]:
response = requests.post(f"{SERVER}/predict", json={"name": "model_rand_2", "X": X, "y": y, "model_type": "randf"})
print(response.status_code, response.text)

200 {"predictions":[1,0,0,1,1,1,0,0,0,1,1,0,1,0,1,1,1,0,0,1,0,1,1,0,1,0,1,1,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,1,1,1,0,0,1,1,1,1,1,1,0,1,0,0,0,1,1,0,1,0,1,0,1,1,1,1,1,1,0,0,1,0,1,1,0,1,0,1,0,0,1,1,0,1,1,0,1,1,1,0,1,0,0,0,0,1,0,0,1,0,1,0,1,0,1,0,1,0,0,1,1,0,0,0,0,1,1,1,0,0,0,1,1,0,1,0,0,1,0,1,1,0,0,1,0,0,1,1,1,0,0,0,0,1,1,1,0,1,0,1,0,0,1,1,0,0,1,1,0,1,1,1,0,0,1,0,1,1,0,0,1,0,1,1,1,1,0,0,1,1,1,1,1,0,1,0,1,0,1,0,1,1,1,1,0,0,0,0,0,1,0,1,1,0,0,1,0,0,0,1,1,1,0,1,1,0,0,0,1,0,0,0,0,0,1,1,0,1,1,0,1,1,0,1,1,0,1,1,1,0,0,0,1,1,0,1,0,0,1,0,0,1,0,1,0,0,0,1,1,0,1,0,0,1,1,0,0,1,1,0,0,0,0,1,1,1,0,1,0,0,1,1,1,0,1,0,0,0,0,0,1,1,0,0,1,0,1,0,1,1,1,1,1,0,1,0,1,0,1,0,1,1,0,1,1,0,1,1,0,0,0,1,1,1,0,1,1,0,1,1,0,1,1,1,1,1,0,1,1,0,1,1,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,1,1,0,0,1,1,1,0,1,0,1,0,1,0,1,0,0,0,1,0,1,0,1,0,0,1,1,1,1,1,0,0,1,0,1,1,0,1,0,1,1,0,1,0,0,0,1,0,1,0,1,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,1,1,1,0,0,0,1,0,1,1,1,0,0,0,1,1,1,1,1,1,1,0,1,1,0,1,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,1,1,1,1,0,0,

Проверка корректности работы load(загрузка уже загруженных моделей)

In [87]:
response = requests.post(f"{SERVER}/load", json={"name": "model_rand_1"})
print(response.text, response.status_code)

{"status":"already_loaded"} 200


In [88]:
response = requests.post(f"{SERVER}/load", json={"name": "model_log_1"})
print(response.text, response.status_code)

{"status":"already_loaded"} 200


Проверка корректности работы load(загрузка больше 3ёх моделей)

In [89]:
response = requests.post(f"{SERVER}/load", json={"name": "model_log_regression_1"})
print(response.text, response.status_code)

{"detail":"You can't load more than 3 models."} 400


Проверка корректности работы unload

In [96]:
response = requests.post(f"{SERVER}/unload", json={"name": "model_rand_2"})
print(response.text, response.status_code)

{"status":"unloaded"} 200


In [97]:
response = requests.post(f"{SERVER}/load", json={"name": "model_log_regression_1"})
print(response.text, response.status_code)

{"status":"loaded"} 200


Проверка корректности работы remove_all

In [91]:
response = requests.post(f"{SERVER}/remove_all")
print(response.text, response.status_code)

{"status":"all_deleted"} 200


In [93]:
response = requests.post(f"{SERVER}/model_log_regression_1", json={"name": "model_rand_2"})
print(response.text, response.status_code)

{"detail":"Not Found"} 404
